In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Bias-Variance tradeoff

$$Y = f(X) + \epsilon \text{ where } \epsilon \sim N(0,\sigma)$$

* The outcome Y is a function of some independent variables X and some error $\epsilon \sim N(0,\sigma_{\epsilon})$
* f(X) is a real-valued function so this is a regression setting
* $\hat{f}(X)$ estimates f(X) (i.e. is a model for f(x))
* The expected prediction error at a point x:
<div style="font-size: 110%;">
$$Err(x) = E[(Y - \hat{f}(x))^2]$$
</div>
* This error may be decomposed:
<div style="font-size: 110%;">
$$Err(x) = (E[\hat{f}(x)] - f(x))^2 + E[(\hat{f}(x) - E[\hat{f}(x)])^2] + \sigma^2_{\epsilon} \\
Err(x) = Bias^2 + Variance + IrreducibleError$$
</div>

* **Bias**: The difference of the models average estimate from the true value
* **Variance**: the average squared difference of the model on a particular training sample and the average over many training samples.

* **Irreducible Error**: The error that can't be reduced even if we had a better model.

![](bias-and-variance.jpg)
$$\text{Figure 1. Visualize Bias and Variance}$$
 
![](model-complexity.png)
$$\text{Figure 2. Bias and Variance Curves}$$

Source:  http://scott.fortmann-roe.com/docs/BiasVariance.html

* To minimize test error need low variance and low bias  
* The Variance is the amount $\hat{f}$ would change if used different training set. 
    - More **complex** (i.e. more parameters, more flexible) methods have higher variance
    - Tuned to the sample, if get a different sample, will get different reducible error
* The Bias is the error introduced by modeling a complicated process with a simpler model
    - Linear regression assumes a linear relationship but most real world problems are not purely linear
    - More complex models have less bias
  
* The error curve is U-shaped error because as we increase complexity (i.e.flexibility) bias tends to decrease faster than variance increases (lower test MSE) but at some point, increasing complexity (i.e. adding more parameters) has little effect on bias but significantly increases variance.

* Where the U-shape curve starts to increase is the point at which **overfitting** occurs

### Model Complexity

* Models with more parameters (degrees of freedom) are more complex
* Simpler models are easier to interpret
    - Occam's razor:  if we have two or more equivalent explanations for the same phenomenon, we should choose the simpler one.
    - But can underfit the data

* Complex models are more **flexible** but are less **interpretable** (not good if inference is the goal)
* If prediction is the goal, more complex models tend to be more accurate but can overfit. 
* We want a model that is high on accuracy but is as simple as possible
    - "Things should be as simple as possible but not simpler" Albert Einstein
  

### Overfitting/Underfitting

* Overfitting: Overfitting occurs when the learning method is finding patterns in the training data caused by random chance (noise) rather than properties of the unknown function f  
    - U-shaped curve  
    - Small training MSE and large test MSE indicates overfitting  
    - Overfitting occurs when **out-of-sample** mean squared error (MSE) increases
 
* Underfitting: When the learning method is not complex enough to accurately capture the
relationships between the response variable and the predictors

#### Within-sample (i.e. training) versus out-of-sample (i.e. test) accuracy

* **Within-sample** accuracy is the accuracy of model fit to the training data. It will always increase with more parameters
* **Out-of-sample** accuracy is the accuracy of the model on new data. When it starts to decrease (error starts to increase) is the point at which it is overfitting the data.


  #### Example 1

In [ ]:
plt.style.use(['seaborn-darkgrid'])
x_n = np.linspace(4,14, 100)
degree = [0, 1, 2, 5]

x = np.array([4.,5.,6., 9.,12, 14.])
y = np.array([4.2, 6., 6., 9., 10, 10.])

def R_squared(y,yhat):
    ybar = np.mean(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2) 
    return np.round(ssreg / sstot,2)

def plot_poly(x,y):
    plt.figure(figsize=(7, 7))
    plt.plot(x, y, 'o')
    for i in degree:
        coeffs = np.polyfit(x, y, deg=i) # numpy polynomial regression
        ffit = np.polyval(coeffs, x_n) 
        poly = np.poly1d(coeffs) # Construct polynomial with coeffs
        yhat = poly(x) # Evaluate
        r2 = R_squared(y,yhat)
        plt.plot(x_n, ffit, label=f'degree {i}, $R^2$= {r2}')
    plt.legend(loc=2, fontsize=14)
    plt.xlabel('$x$', fontsize=14)
    plt.ylabel('$y$', fontsize=14, rotation=0)

plot_poly(x,y)

* The model increases in complexity as more terms (parameters) are added and $R^2$ increases explaining more of the variance in y

#### Overfitting
 
* Model with degree 5 fits the data perfectly
    - Same number of parameters as data points
    - Notice the erratic behavior of this model between the data points (9,9) and (12,9).
* The behavior of degree 2 model is much smoother, it predicts a parabola with some noise
    - This seems a more reasonable model even though it has a lower $R^2$
* If we collected more data points, say (10,9) and (7,7)

In [ ]:
x = np.array([4.,5.,6., 7, 9.,10,12, 14.])
y = np.array([4.2, 6., 6., 7, 9., 9, 10, 10.])
plot_poly(x,y)

* Notice how the degree 5 model changed shape and its $R^2$ went down.
* Degree 1 and 2 models changed very little. They were better predictors of the new data
* Models that overfit the data have high variance.

#### Underfitting

* Model with degree 0 has no coefficients so the model is just a model of the dependent variable alone. The orange line represents the mean of the dependent variable y. It is not capturing any patterns in the data.
* Models that underfit the data, like the degree 0 and degree 1 models, have high bias



### Overfitting Example 2

In [ ]:
from sklearn.linear_model import LinearRegression
np.random.seed(42)
N = 10 # 10 Polynomials, degree 1 - 10

# Generate some noisy nonlinear data

x = np.array([i*np.pi/180 for i in range(60,300,4)]) # radians 60-300 degrees
y = np.sin(x) + np.random.normal(0,0.15,len(x)) # Add some noise to sin function
print(x.shape,y.shape)
plt.plot(x,y,'o');

#### Create an array where rows are the inputs and the columns are polynomials $x^1$ to $x^{10}$

In [ ]:
X = np.array([x**(n+1) for n in range(N)]).T
# Rows of Nth degree polynomial for 60 data points on the x-axis
print(X[0,:])
X.shape,y.shape

#### N Polynomial Regression models

* Accumulate model coefficients, predictions and mean squared error

In [ ]:
models = [LinearRegression() for i in range(N)]
coefs = np.zeros((N,N))
yhats = []
mses = np.zeros(N)
scores = np.zeros(N)
r2 = np.zeros(N)
for i in range(N):
    models[i].fit(X[:,0:(i+1)], y)
    yhats.append(models[i].predict(X[:,0:(i+1)]))
    for j in range(len(models[i].coef_)):
        coefs[i,j] = models[i].coef_[j]
    mses[i] = np.mean((y - yhats[-1])**2)
    

#### Print coefficient matrix

In [ ]:
print('Deg  Coefficients')
for i in range(N):
    a = [x for x in coefs[i]]
    print(i+1, list(map(lambda x: round(x,3), a)))

#### Plot Mean Squared Error

In [ ]:
plt.plot(range(len(mses)),mses)
plt.title("MSE")
plt.ylabel("MSE")
plt.xlabel("Model");

#### Plot data and regression line for models 1,4,7,10

In [ ]:
fig,((ax0,ax1),(ax2,ax3)) = plt.subplots(2,2,figsize = (12,6))
axs = [ax0,ax1,ax2,ax3]
for i,ax in enumerate(axs):
    ax.plot(x,y,'o')
    ax.plot(x,yhats[i*3])
plt.tight_layout()
    

#### Create some test data

In [ ]:
np.random.seed(1234) # Using a different seed
x = np.array([i*np.pi/180 for i in range(60,300,4)])
y = np.sin(x) + np.random.normal(0,0.15,len(x))
X = np.array([x**(n+1) for n in range(N)]).T

In [ ]:
coefs = np.zeros((N,N))
yhats = []
mses_test = np.zeros(N)
for i in range(N):
    models[i].fit(X[:,0:(i+1)], y)
    yhats.append(models[i].predict(X[:,0:(i+1)]))
    for j in range(len(models[i].coef_)):
        coefs[i,j] = models[i].coef_[j]
    mses_test[i] = np.mean((y - yhats[-1])**2)

In [ ]:
p1 = plt.plot(range(len(mses)),mses)
p2 = plt.plot(range(len(mses_test)),mses_test)
plt.ylabel("MSE")
plt.legend((p1[0],p2[0]),("Training","Test"));

#### Summary

* Higher MSE for new data
* Magnitude of coefficients increased with model complexity